<a href="https://colab.research.google.com/github/saranshtyagi/generative-adversial-networks/blob/main/GANs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy  as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
!wget http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
!wget http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz
!wget http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz
!wget http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz

--2024-10-03 13:00:15--  http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
Resolving fashion-mnist.s3-website.eu-central-1.amazonaws.com (fashion-mnist.s3-website.eu-central-1.amazonaws.com)... 3.5.136.48, 3.5.134.113, 3.5.135.170, ...
Connecting to fashion-mnist.s3-website.eu-central-1.amazonaws.com (fashion-mnist.s3-website.eu-central-1.amazonaws.com)|3.5.136.48|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26421880 (25M) [binary/octet-stream]
Saving to: ‘train-images-idx3-ubyte.gz’

train-images-idx3-u 100%[===================>]  25.20M   103MB/s    in 0.2s    

2024-10-03 13:00:15 (103 MB/s) - ‘train-images-idx3-ubyte.gz’ saved [26421880/26421880]

--2024-10-03 13:00:16--  http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz
Resolving fashion-mnist.s3-website.eu-central-1.amazonaws.com (fashion-mnist.s3-website.eu-central-1.amazonaws.com)... 3.5.136.48, 3.5.134.113, 3.5.135.170, ...
Conn

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
# Training params
learning_rate = 0.0002
batch_size = 128
epochs = 100000

# Network params
image_dim = 784   # 28x28 image size flattened
gen_hidd_dim = 256  # Generator hidden layer dimension
disc_hidd_dim = 256 # Discriminator hidden layer dimension
z_noise_dim = 100   # Input noise dimension for the generator

# Xavier initializer function
def xavier_init(shape):
    initializer = tf.keras.initializers.GlorotNormal()
    return initializer(shape)

In [ ]:
# Define weights and bias dictionaries using Xavier initialization
weights = {
    "disc_H": tf.Variable(xavier_init([image_dim, disc_hidd_dim]), trainable=True),
    "disc_final": tf.Variable(xavier_init([disc_hidd_dim, 1]), trainable=True),
    "gen_H": tf.Variable(xavier_init([z_noise_dim, gen_hidd_dim]), trainable=True),
    "gen_final": tf.Variable(xavier_init([gen_hidd_dim, image_dim]), trainable=True)
}

bias = {
    "disc_H": tf.Variable(xavier_init([disc_hidd_dim]), trainable=True),
    "disc_final": tf.Variable(xavier_init([1]), trainable=True),
    "gen_H": tf.Variable(xavier_init([gen_hidd_dim]), trainable=True),
    "gen_final": tf.Variable(xavier_init([image_dim]), trainable=True)
}

In [ ]:
# Define Discriminator function
def Discriminator(x):
    hidden_layer = tf.nn.relu(tf.add(tf.matmul(x, weights["disc_H"]), bias["disc_H"]))
    final_layer = tf.add(tf.matmul(hidden_layer, weights["disc_final"]), bias["disc_final"])
    disc_output = tf.nn.sigmoid(final_layer)
    return final_layer, disc_output

# Define generator network
def Generator(x):
    hidden_layer = tf.nn.relu(tf.add(tf.matmul(x, weights["gen_H"]), bias["gen_H"]))
    final_layer = tf.add(tf.matmul(hidden_layer, weights["gen_final"]), bias["gen_final"])
    gen_output = tf.nn.sigmoid(final_layer)
    return gen_output

# Input noise for the generator and real images for the discriminator
z_input = tf.random.normal([batch_size, z_noise_dim], name="input_noise")  # Generator input (noise)
x_input = tf.random.normal([batch_size, image_dim], name="real_input")     # Discriminator input (real images)

# Building the Generator Network
with tf.name_scope("Generator"):
    output_Gen = Generator(z_input)

# Building the Discriminator Network
with tf.name_scope("Discriminator"):
    real_output1_Disc, real_output_Disc = Discriminator(x_input)        # Real data passed to discriminator
    fake_output1_Disc, fake_output_Disc = Discriminator(output_Gen)     # Fake data from generator passed to discriminator


In [ ]:
# First kind of loss

with tf.name_scope("Discriminator Loss") as scope:
    Discriminator_Loss = -tf.reduce_mean(tf.math.log(real_output_Disc + 0.0001) + tf.math.log(1. - fake_output_Disc + 0.0001))

with tf.name_scope("Generator Loss") as scope:
    Generator_Loss = -tf.reduce_mean(tf.math.log(fake_output_Disc + 0.0001))


In [ ]:
# Second type of loss
with tf.name_scope("Discriminator_Loss") as scope:
    Disc_real_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=real_output1_Disc, labels=tf.ones_like(real_output1_Disc)))
    Disc_fake_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=fake_output1_Disc, labels=tf.zeros_like(fake_output1_Disc)))
    Discriminator_Loss = Disc_real_loss + Disc_fake_loss

with tf.name_scope("Generator_Loss") as scope:
    Generator_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=fake_output1_Disc, labels=tf.ones_like(fake_output1_Disc)))

# TensorBoard Summary setup
summary_writer = tf.summary.create_file_writer("logs/")

# Add summaries to the writer inside a tf.summary context
with summary_writer.as_default():
    tf.summary.scalar("Discriminator_Loss/Real", Disc_real_loss, step=1)
    tf.summary.scalar("Discriminator_Loss/Fake", Disc_fake_loss, step=1)
    tf.summary.scalar("Discriminator_Loss/Total", Discriminator_Loss, step=1)
    tf.summary.scalar("Generator_Loss", Generator_Loss, step=1)

In [ ]:
# Define the variables
Generator_var = [weights["gen_H"], weights["gen_final"], bias["gen_H"], bias["gen_final"]]
Discriminator_var = [weights["disc_H"], weights["disc_final"], bias["disc_H"], bias["disc_final"]]

# Define the optimizer for Discriminator and Generator
Discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
Generator_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)


In [ ]:
# Training loop using GradientTape
for epoch in range(epochs):
    x_batch = np.random.normal(size=[batch_size, image_dim]).astype(np.float32)
    z_noise = np.random.uniform(-1., 1., size=[batch_size, z_noise_dim]).astype(np.float32)

    with tf.GradientTape() as disc_tape, tf.GradientTape() as gen_tape:
        real_output1_Disc, real_output_Disc = Discriminator(x_batch)
        fake_output1_Disc, fake_output_Disc = Discriminator(Generator(z_noise))

        # Compute losses
        Disc_real_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=real_output1_Disc, labels=tf.ones_like(real_output1_Disc)))
        Disc_fake_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=fake_output1_Disc, labels=tf.zeros_like(fake_output1_Disc)))
        Discriminator_Loss = Disc_real_loss + Disc_fake_loss
        Generator_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=fake_output1_Disc, labels=tf.ones_like(fake_output1_Disc)))

    # Compute gradients
    Discriminator_gradients = disc_tape.gradient(Discriminator_Loss, Discriminator_var)
    Generator_gradients = gen_tape.gradient(Generator_loss, Generator_var)

    # Apply gradients
    Discriminator_optimizer.apply_gradients(zip(Discriminator_gradients, Discriminator_var))
    Generator_optimizer.apply_gradients(zip(Generator_gradients, Generator_var))

    # Print the progress
    if epoch % 2000 == 0:
        print("Epoch: {0}, Generator Loss: {1}, Discriminator Loss: {2}".format(epoch, Generator_loss.numpy(), Discriminator_Loss.numpy()))


Epoch: 0, Generator Loss: 0.30472517013549805, Discriminator Loss: 1.8519678115844727
Epoch: 2000, Generator Loss: 4.121630668640137, Discriminator Loss: 0.01910434290766716
Epoch: 4000, Generator Loss: 6.169473648071289, Discriminator Loss: 0.0025293815415352583
Epoch: 6000, Generator Loss: 7.583927154541016, Discriminator Loss: 0.0006175156449899077
Epoch: 8000, Generator Loss: 9.423749923706055, Discriminator Loss: 0.00011797028128057718
Epoch: 10000, Generator Loss: 9.966355323791504, Discriminator Loss: 6.261878297664225e-05
Epoch: 12000, Generator Loss: 11.090300559997559, Discriminator Loss: 1.8123817426385358e-05
Epoch: 14000, Generator Loss: 8.839189529418945, Discriminator Loss: 0.0002616965211927891
Epoch: 16000, Generator Loss: 10.557866096496582, Discriminator Loss: 2.6256362616550177e-05
Epoch: 18000, Generator Loss: 8.586808204650879, Discriminator Loss: 0.0001912212319439277
Epoch: 20000, Generator Loss: 10.697322845458984, Discriminator Loss: 2.2826829081168398e-05
Epo

In [ ]:
# Create a TensorFlow session
sess = tf.Session()
sess.run(tf.global_variables_initializer())  # Initialize all variables

# Testing
# Generate images from noise, using the generator network.
n = 6
canvas = np.empty((28 * n, 28 * n))
for i in range(n):
  # Noise input
  Z_noise = np.random.uniform(-1.,1., size = [batch_size, z_noise_dim])
  # Generate image from noise.
  g = sess.run(output_Gen, feed_dict = {z_input : z_noise})
  # Reverse the colours for better display
  g = -1 * (g - 1)
  for j in range(n):
    # Draw the generated digits
    canvas[i * 28 : (i + 1) * 28, j * 28 : (j + 1) * 28] = g[j].reshape([28, 28])
plt.figure(figsize = (n, n))
plt.imshow(canvas, origin = "upper", cmap = "gray")
plt.show()